# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression?
Why?

#####Response:
This is a classification problem since we have to predict if a student does need early intervention or not.

Our predictor will output just two discere classes: Ture or False.
* True: student will graduate -> No need for early intervention
* False: student will not graduate -> Needs early intervention

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [24]:
# Import libraries
import numpy as np
import pandas as pd

In [25]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [26]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1  # Subtract the target column
n_passed = sum(student_data['passed'] == 'yes')
n_failed = sum(student_data['passed'] == 'no')
grad_rate = (float(n_passed) / n_students) * 100.0
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


#####Observation
Good that the dataset does not appear to be much squwed. (i.e.: we do not have a huge predominance of one of the two classes)

## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [27]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [28]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [29]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size = num_train)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

##Decision Tree Model

In [30]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier
    
clf_DT = DecisionTreeClassifier(max_depth= 2)

# Fit model to training data
train_classifier(clf_DT, X_train, y_train)  # note: using entire training set here
print clf_DT  # you can inspect the learned model by printing it
print "Relevant features: {}".format(list(X_all.columns[clf_DT.feature_importances_ > 0.0]))

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
Relevant features: ['failures', 'schoolsup', 'freetime']


In [31]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf_DT, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.824833702882


In [32]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_DT, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.80303030303


In [33]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

### Prepare train data for 100 and 200 samples.
### Train data with 300 samples is already the full train set anyway we set it
### Theese sets will be also used by subsequent models
X_100 = X_train[0:100]
y_100 = y_train[0:100]
X_200 = X_train[0:200]
y_200 = y_train[0:200]
X_300 = X_train[0:300]
y_300 = y_train[0:300]


train_predict(clf_DT, X_100, y_100, X_test, y_test)
train_predict(clf_DT, X_200, y_200, X_test, y_test)
train_predict(clf_DT, X_300, y_300, X_test, y_test)


------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.007
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.820895522388
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.745762711864
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.838283828383
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.790697674419
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 sc

#####Observations
The test F1 score might have some down-up fluctuation while training on 100, 200 and then 300 samples. This might be caused by the samples disposition in the train subsets. Anyway for the 300 samples train the test score is consistent with the train score, telling us we are not badly overfitting.

Since the maximum depth is pretty low we can say that just a few features can be good at predicting if the student needs help:

In [34]:
print "Relevant features: {}".format(list(X_all.columns[clf_DT.feature_importances_ > 0.0]))

Relevant features: ['failures', 'schoolsup', 'freetime']


Theese relevant features found by training the decision tree always inlude `failures` which is common sense. Other relevant features vary a lot due to randomisation. This is a hint that `failures` is really a strong indicator, and we can probably further reduce model complexity (i.e.: max tree depth). More on this in the digression at the end of the project.

##Gaussian Naive Bayes Model

In [35]:
# TODO: Choose a model, import it and instantiate an object
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB()

# Fit model to training data
train_classifier(clf_NB, X_train, y_train)  # note: using entire training set here
print clf_NB  # you can inspect the learned model by printing it

Training GaussianNB...
Done!
Training time (secs): 0.002
GaussianNB()


In [36]:
train_f1_score = predict_labels(clf_NB, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.77694235589


In [37]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_NB, X_test, y_test))

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.793650793651


In [38]:
train_predict(clf_NB, X_100, y_100, X_test, y_test)
train_predict(clf_NB, X_200, y_200, X_test, y_test)
train_predict(clf_NB, X_300, y_300, X_test, y_test)
nb_params = clf_NB.get_params()
print len(nb_params)

------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.285714285714
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.325
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.005
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.756554307116
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.762711864407
------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.002
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.77694235589
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0

##Support Vector Machine Model

In [39]:
# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import SVC
clf_SVC = SVC(C = 0.8)

# Fit model to training data
train_classifier(clf_SVC, X_train, y_train)  # note: using entire training set here
print clf_SVC  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.019
SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [40]:
train_f1_score = predict_labels(clf_SVC, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.012
F1 score for training set: 0.831275720165


In [41]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_SVC, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.810810810811


In [42]:
train_predict(clf_SVC, X_100, y_100, X_test, y_test)
train_predict(clf_SVC, X_200, y_200, X_test, y_test)
train_predict(clf_SVC, X_300, y_300, X_test, y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.846153846154
Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for test set: 0.82119205298
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.007
Predicting labels using SVC...
Done!
Prediction time (secs): 0.011
F1 score for training set: 0.845921450151
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.81045751634
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.012
Predicting labels using SVC...
Done!
Prediction time (secs): 0.012
F1 score for training set: 0.831275720165
Predicting labels using SVC...
Done!
Prediction time (secs): 0.012
F1 score for test set: 0.810810810811


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [43]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score


from sklearn.grid_search import GridSearchCV

parameters = {'C':[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2],
              'kernel': [ 'linear', 'poly', 'rbf', 'sigmoid'],
              'gamma': [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08],
              'degree': [2, 3, 4, 5, 6],             # degree of polynomial kernel, ignored by other kernels
              'decision_function_shape': ['ovr']     # one vs. the rest. (none and one vs. one will be deprecated)
             }
scorer = make_scorer(f1_score, greater_is_better=True, pos_label = 'yes')
selectedModelSpace = SVC()

searcher = GridSearchCV(selectedModelSpace, parameters, scorer)
searcher.fit(X_train, y_train)
print searcher.best_score_
print searcher.best_estimator_

0.825512852802
SVC(C=0.9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma=0.06, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [44]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(searcher.best_estimator_, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.805369127517


#Digressions

###GridSearch on Decision Tree

In [45]:
parameters_DT = {'max_depth':[1, 2, 3, 4, 5, 6], 'min_samples_split': [2, 3, 4, 5, 10, 20]}

dt_ModelSpace = DecisionTreeClassifier()

searcher_DT = GridSearchCV(dt_ModelSpace, parameters_DT, scorer)
searcher_DT.fit(X_train, y_train)
print searcher_DT.best_score_
best_DT = searcher_DT.best_estimator_
print best_DT
print "Relevant features: {}".format(list(X_all.columns[best_DT.feature_importances_ > 0.0]))
print best_DT.tree_

0.811128206849
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
Relevant features: ['failures']


In [46]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(best_DT, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.790697674419


_Decision Tree fine tuneing always gives a slightly lower final F1 Score._